<a href="https://colab.research.google.com/github/Terunori/signate-578/blob/main/%5BSignate%5D_578_%E6%90%BA%E5%B8%AF%E9%9B%BB%E8%A9%B1%E4%BE%A1%E6%A0%BC%E5%B8%AF%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

【第21回_Beginner限定コンペ】携帯電話の機能データからの価格帯分類
携帯電話の機能から販売価格を分類しよう！ 

https://signate.jp/competitions/578

links
- [ニューラルネットワークの実装（基礎）](https://free.kikagaku.ai/tutorial/basic_of_deep_learning/learn/pytorch_basic)
- [Pytorchによる多クラス分類の実装](https://venoda.hatenablog.com/entry/2020/10/03/075322#42-%E6%90%8D%E5%A4%B1%E9%96%A2%E6%95%B0%E3%81%AE%E5%AE%9A%E7%BE%A9)
- [多クラス分類におけるF値](https://zenn.dev/hellorusk/articles/46734584386c49057e1b)

In [ ]:
# GPUの状態確認
! nvidia-smi

# 起動時設定
from datetime import datetime, timezone, timedelta

tz_jst = timezone(timedelta(hours=9))
connect_time = datetime.now(tz=tz_jst).strftime('%Y%m%d_%H%M')

# Google Drive マウント
from google.colab import drive
import sys
drive.mount('/content/drive')

# signate connect
sys.path.append('/content/drive/MyDrive/Colab Notebooks/modules')
import signateConnect

! pip install signate

signateConnect.connect()

! signate files --competition-id=578

In [ ]:
! signate download --competition-id=578

In [ ]:
import pandas as pd

# idは学習データに含みたくないので除外
df_train = pd.read_csv('train.csv').drop('id', axis=1)

In [ ]:
df_train.head(3)

In [ ]:
import torch
from sklearn.preprocessing import StandardScaler

# 正規化
scaler = StandardScaler()
scaler.fit(df_train.drop('price_range', axis=1))

# 入力と出力に分解
y = torch.LongTensor(df_train['price_range'].values)
x = torch.Tensor(df_train.drop('price_range', axis=1).values)

# デフォルトのデータセット定義を使用
dataset = torch.utils.data.TensorDataset(x,y)

In [ ]:
batch_size = 64

In [ ]:
# train_rate = .87
train_rate = .65

train_dataset, valid_dataset = torch.utils.data.random_split(
    dataset, 
    [int(len(dataset)*train_rate), int(len(dataset)*(1-train_rate))]
)

In [ ]:
# loader

# 学習用Dataloader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,
    drop_last=True,
    pin_memory=True
)

# 評価用Dataloader
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset, 
    batch_size=batch_size, 
    shuffle=False,
    drop_last=True,
    pin_memory=True
)

In [ ]:
# 動作確認
batch_iterator = iter(train_dataloader)
# 1番目の要素を取り出す
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels.size())
print(inputs[0])
print(labels[0])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        inFeatures  = 20  # 入力層サイズ  : 入力特徴量の数
        mid1Features = 48 # 中間層1サイズ : はじめに入力より大きくする
        mid2Features = 32 # 中間層2サイズ : だんだん小さくする
        mid3Features = 16 # 中間層3サイズ
        outFeatures = 4   # 出力層サイズ  :  分類したいクラス数

        self.fc1 = nn.Linear(inFeatures, mid1Features)
        self.bn1 = nn.BatchNorm1d(mid1Features)
        self.fc2 = nn.Linear(mid1Features, mid2Features)
        self.bn2 = nn.BatchNorm1d(mid2Features)
        self.fc3 = nn.Linear(mid2Features, mid3Features)
        self.bn3 = nn.BatchNorm1d(mid3Features)
        self.fc4 = nn.Linear(mid3Features, outFeatures)

    def forward(self, x):     # 一次元なので全結合型DNN
        x = self.fc1(x)       # 全結合層 KerasやNeural Network ConsoleではDense層, Affine層とも呼ばれる
        x = self.bn1(x)       # Batch normalization層 勾配消失・爆発を防ぐ効果: → learning rateを大きくしても収束しやすくなる
        x = F.relu(x)         # 活性化 ReLu使用
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, 0.2) # Dropout層
        x = self.fc3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.softmax(x, dim=1)
        return x

net = Net()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)
print(net)

In [ ]:
# loss クロスエントロピー
criterion = nn.CrossEntropyLoss()

In [ ]:
# 最適化手法
import torch.optim as optim

# Adam使用するとSGDより学習が速い(が、過学習もしやすい)
learningRate = 1e-3
optimizer = optim.Adam(net.parameters(), lr=learningRate)

In [ ]:
# エポック数
num_epochs = 2500

# 学習時と検証時で分けるためディクショナリを用意
dataloaders_dict = {
    'train': train_dataloader,
    'val': valid_dataloader
}

# best
best_f1 = 0
best_net = 0

# log
epoch_log = 200

for epoch in range(num_epochs):
    
    if epoch % epoch_log == epoch_log-1:
        print()
        print(f'epoch: {epoch+1:4}')

    for phase in ['train', 'val']:
        
        if phase == 'train':
            # モデルを訓練モードに設定
            net.train()
        else:
            # モデルを推論モードに設定
            net.eval()
        
        # 損失和
        epoch_loss = 0.0
        # 正解数
        epoch_corrects = 0
        # f1macro
        f1 = []
        f1_macro = 0
        
        # DataLoaderからデータをバッチごとに取り出す
        for inputs, labels in dataloaders_dict[phase]:

            # GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # optimizerの初期化
            optimizer.zero_grad()
            
            # 学習時のみ勾配を計算させる設定にする
            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)
                
                # 損失を計算
                loss = criterion(outputs, labels)
                
                # ラベルを予測
                _, preds = torch.max(outputs, 1)
                
                # 訓練時はバックプロパゲーション
                if phase == 'train':
                    # 逆伝搬の計算
                    loss.backward()
                    # パラメータの更新
                    optimizer.step()
                
                # イテレーション結果の計算
                # lossの合計を更新
                # PyTorchの仕様上各バッチ内での平均のlossが計算される。
                # データ数を掛けることで平均から合計に変換をしている。
                # 損失和は「全データの損失/データ数」で計算されるため、
                # 平均のままだと損失和を求めることができないため。
                epoch_loss += loss.item() * inputs.size(0)
                
                # 正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)

                # F値計算 毎回やると重いので適当な頻度で実施
                if epoch % 20 == 19:
                    for i in range(4):
                        tp, fp, fn, tn = 0,0,0,0
                        for pred, fact in zip(preds, labels.data):
                            if pred == fact == i:
                              tp+=1
                            elif i == pred:
                              fp+=1
                            elif i == fact:
                              fn+=1
                            else:
                              tn+=1
                        
                        if tp == 0:
                            f1.append(0)
                        else:
                            precision = tp/(tp+fp)
                            recall = tp/(tp+fn)
                            f1.append(2*precision*recall/(precision+recall))

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
        # 今回の評価指標であるf2macroを計算(f1が計算されたときのみ有効値でそれ以外だと0になる)
        f1_macro = np.mean(f1)

        if phase != 'train':
            if f1_macro > best_f1: # 評価指標に対し最も良い状態の保存
                best_net = net
                best_f1 = f1_macro

        if epoch % epoch_log == epoch_log-1:
            print('{} Loss: {:.4f} Acc: {:.4f} F1: {:,.4f} Best: {:,.4f}'.format(phase, epoch_loss, epoch_acc, f1_macro, best_f1))



In [ ]:
# 最終結果の保存
# 今回はtestデータを推論した結果のcsvを送信するため直接必要なわけではない
torch.save(net.state_dict(), './signate578_' +
           str(epoch+1) + '.pth')

In [ ]:
# 学習のvalidデータで確認
f1 = []
for inputs, labels in valid_dataloader:
    # GPU
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    outputs = best_net(inputs)
    
    # ラベルを予測
    _, preds = torch.max(outputs, 1)
        
    # 正解数の合計を更新
    epoch_corrects += torch.sum(preds == labels.data)

    # F値計算
    for i in range(4):
        tp, fp, fn, tn = 0,0,0,0
        for pred, fact in zip(preds, labels.data):
          if pred == fact == i:
            tp+=1
          elif i == pred:
            fp+=1
          elif i == fact:
            fn+=1
          else:
            tn+=1
        
        if tp == 0:
          f1.append(0)
        else:
          precision = tp/(tp+fp)
          recall = tp/(tp+fn)
          f1.append(2*precision*recall/(precision+recall))

# epochごとのlossと正解率を表示
epoch_acc = epoch_corrects.double() / len(valid_dataloader.dataset)
f1_macro = np.mean(f1)

print(epoch_acc, f1_macro)


In [ ]:
# 提出用CSVデータ作成
import numpy as np

# 推論
df_test = pd.read_csv('test.csv')

df_test_test = df_test.drop('id', axis=1)

scaler.fit(df_test_test)
test = torch.Tensor(df_test_test.values).to(device)

out = best_net(test)
_, preds = torch.max(out, 1)

np_pred = preds.cpu().detach().numpy().astype(np.int64)

df_test['result'] = pd.Series(np_pred)
df_result = df_test.loc[:,['id','result']]

df_result.to_csv('result.csv', sep=',', header=False, index=False)
